**Step 1:** Import required modules.

In [18]:
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.examples.tutorials.mnist import input_data

**Step 2**: Only show ERROR logs.

In [19]:
tf.logging.set_verbosity(tf.logging.ERROR)

**Step 3:** Load dataset.

In [20]:
dataset = input_data.read_data_sets("./MNIST-data", one_hot=True)

Extracting ./MNIST-data/train-images-idx3-ubyte.gz


Extracting ./MNIST-data/train-labels-idx1-ubyte.gz
Extracting ./MNIST-data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST-data/t10k-labels-idx1-ubyte.gz


**Step 4:** Show available devices.

In [21]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 718513500886815693, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 720896
 locality {
   bus_id: 1
 }
 incarnation: 6321069589334856114
 physical_device_desc: "device: 0, name: GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"]

**Step 5:** Create model.

In [22]:
x = tf.placeholder(tf.float32, [None, 784], "TrainSet")

with tf.name_scope("WeightsMatrix"):
    W = tf.Variable(tf.zeros([784, 10]))

with tf.name_scope("AdjustVector"):
    b = tf.Variable(tf.zeros([10]))

y = tf.matmul(x, W) + b

**Step 6:** Define loss and optimizer.

In [23]:
y_ = tf.placeholder(tf.float32, [None, 10], "TrainLabels")

**Step 7:** Calculate cross entropy.

In [24]:
with tf.name_scope("Softmax"):
    softmax = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)

with tf.name_scope("Lost"):
    loss = tf.reduce_mean(softmax)

with tf.name_scope("Optimizer"):
    optimizer = tf.train.GradientDescentOptimizer(0.01) 
    train = optimizer.minimize(loss)

**Step 8:** Initialize session.

In [25]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

**Step 8:** Determine algorithm accuracy.

In [26]:
with tf.name_scope("Accurary"):
    # Return a Boolean array where a occuency is true when both(y, y_) are equal to each other.
    precision = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(precision, tf.float32))

**Step 9:** Define accuracy, cost, train and show train step functions.

In [27]:
def get_accuracy(sess, validation_features, validation_labels):
    return sess.run(accuracy, feed_dict={x:validation_features, y_: validation_labels})

In [28]:
def get_loss(sess, features, labels):
    return sess.run(loss, feed_dict={x: features, y_: labels})

In [29]:
def train_model(sess, train_features, train_labels):
    sess.run(train, feed_dict={x: train_features, y_: train_labels})

In [30]:
def show_train_step_info(sess, step, train_features, train_labels, every_n_steps = 2000):
    if (step%every_n_steps!=0): return
    current_loss = get_loss(sess, train_features, train_labels)
    current_accuracy = get_accuracy(sess, dataset.validation.images, dataset.validation.labels)
    print('Step: {:<4} - Loss: {:<8.3} Accuracy: {:<5.3}%'.format(step, current_loss, current_accuracy*100))

**Step 10:** Train model 5000 times.

In [31]:
for step in range(32000):
  features, labels = dataset.train.next_batch(100)
  train_model(sess, features, labels)
  show_train_step_info(sess, step, features, labels)

Step: 0    - Loss: 2.29     Accuracy: 44.8 %


Step: 2000 - Loss: 0.642    Accuracy: 88.3 %


Step: 4000 - Loss: 0.321    Accuracy: 89.5 %


Step: 6000 - Loss: 0.268    Accuracy: 90.3 %


Step: 8000 - Loss: 0.386    Accuracy: 90.8 %


...

Step: 22000 - Loss: 0.358    Accuracy: 91.8 %


Step: 24000 - Loss: 0.271    Accuracy: 92.0 %


Step: 26000 - Loss: 0.474    Accuracy: 92.1 %


Step: 28000 - Loss: 0.382    Accuracy: 92.1 %


Step: 30000 - Loss: 0.316    Accuracy: 92.2 %


**Step 11:** Determine model accuracy.

In [32]:
def saveGraph(sess):
    tf.summary.FileWriter('./logs', graph_def=sess.graph_def)

In [33]:
saveGraph(sess)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

result = sess.run(accuracy, feed_dict={x: dataset.test.images, y_: dataset.test.labels})

print(f"Accurary: {result*100}%")


Accurary: 91.94999933242798%


**Step 12:** Perform a test.